# W5-CRISPR-Cas9-inframe_deletion

In Workflow 5, CRISPR-Cas9 combines ssDNA bridging and Gibson cloning to create in-frame deletions (Figure 6A). This ensures that the downstream genes are not disrupted, which is crucial for functional studies of gene knockouts. This workflow can be used to study gene function by knocking out specific genes in streptomycetes and observing the resulting phenotypic changes. It can also be used to create markerless deletions, which are important for constructing clean genetic backgrounds for further genetic manipulations. To get started, download the pCRISPR-Cas9 file and a genome file (e.g., S. coelicolor A3), then select the genes to delete. StreptoAIM will generate the necessary primers and plasmid constructs (S5).


In [1]:
import sys
import os
from Bio.Restriction import StuI
from pydna.dseqrecord import Dseqrecord
import pandas as pd
from datetime import datetime


# Ensure the src directory is in the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from streptocad.sequence_loading.sequence_loading import (
    load_and_process_gene_sequences,
    load_and_process_genome_sequences,
    load_and_process_plasmid, 
    check_and_convert_input,
    annotate_dseqrecord,
    process_specified_gene_sequences_from_record)

from streptocad.utils import polymerase_dict, create_primer_df_from_dict,ProjectDirectory,extract_metadata_to_dataframe
from streptocad.primers.primer_generation import create_idt_order_dataframe
from streptocad.cloning.ssDNA_bridging import assemble_plasmids_by_ssDNA_bridging,make_ssDNA_oligos
from streptocad.crispr.guideRNAcas3_9_12 import extract_sgRNAs, SgRNAargs
from streptocad.cloning.gibson_cloning import (
    find_up_dw_repair_templates,
    assemble_multiple_plasmids_with_repair_templates_for_deletion,
    update_primer_names
)

from streptocad.cloning.plasmid_processing import check_plasmid_restriction_sites, determine_workflow_order_for_plasmids
from streptocad.primers.primer_generation import checking_primers, primers_to_IDT, find_best_check_primers_from_genome


## INPUT

In [2]:
# Inputs
# 1 Add genome of choice (genbank, fasta)
path_to_genome = '../../data/genomes/Streptomyces_coelicolor_A3_chromosome.gb'
genome = load_and_process_genome_sequences(path_to_genome)[0]

# 2 Add plasmid 
path_to_plasmid = '../../data/plasmids/pCRISPR–Cas9_plasmid_addgene.gbk'
clean_plasmid = load_and_process_plasmid(path_to_plasmid)

# 3 Choose genes to knock out (list)
#genes_to_KO = ['SCO5087', 'SCO5089', 'SCO5090','SCO5091', 'SCO5092' ]
genes_to_KO = ['80000-100000', '4000-7000', '9000-14000','15000-20000']


#### Advanced settings ####
# 4 Filtering metrics for sgRNAs
gc_upper = 0.72
gc_lower = 0.2
off_target_seed = 13
off_target_upper = 10
cas_type='cas9'
number_of_sgRNAs_per_group = 2

# 5 Choose polymerase and target melting temperature
chosen_polymerase = polymerase_dict['Phusion High-Fidelity DNA Polymerase (GC Buffer)']
melting_temperature = 65
primer_concentration = 0.4 
primer_number_increment = 23
flanking_region = 500

# 6 Choose overlapping sequences for our plasmid we can use the following
#As per the article **"CRISPR–Cas9, CRISPRi and CRISPR-BEST-mediated genetic manipulation in streptomycetes"** we need the following oligoes: 
#CGGTTGGTAGGATCGACGGC **-N20-** GTTTTAGAGCTAGAAATAGC
up_homology = Dseqrecord('CGGTTGGTAGGATCGACGGC')
dw_homology = Dseqrecord('GTTTTAGAGCTAGAAATAGC')

# 7 extra settings
#### Advanced settings ####
repair_length = 1000
overlap_length = 40 

In [3]:
print(clean_plasmid.id)

pCRISPR-Cas9


# Computation

In [4]:
target_dict, genes_to_KO, annotation_input = check_and_convert_input(genes_to_KO)

print(annotation_input)
if annotation_input == True:
    genome = annotate_dseqrecord(genome, target_dict)


len(genome.features)


True
this is the target dict [{'region1': [80000, 100000]}, {'region2': [4000, 7000]}, {'region3': [9000, 14000]}, {'region4': [15000, 20000]}]
region1 [80000, 100000]
region2 [4000, 7000]
region3 [9000, 14000]
region4 [15000, 20000]


25828

In [5]:

# Initialize SgRNAargs with desired parameters
args = SgRNAargs(genome, 
                genes_to_KO,
                step=['find', 'filter'],
                gc_upper = gc_upper,
                gc_lower = gc_lower,
                off_target_seed = off_target_seed,
                off_target_upper = off_target_upper,
                cas_type='cas9'
                )

sgrna_df = extract_sgRNAs(args)
sgrna_df

sgRNA generated were outside the designated border in region1. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in region1. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in region2. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in region2. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in region2. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in region2. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in region2. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in region2. To incorporate this extent borders. Skipping to 

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
3517,NC_003888.3,region1,80001,1,1,6574,0.60,TGG,GCCACATTCACTACGACCCC,TCACTACGACCCC,0
2538,NC_003888.3,region1,80001,1,1,16046,0.65,GGG,CCCTGATCCGAGTGTGTGCC,CCGAGTGTGTGCC,0
2539,NC_003888.3,region1,80001,1,1,16045,0.60,CGG,ACCCTGATCCGAGTGTGTGC,TCCGAGTGTGTGC,0
2540,NC_003888.3,region1,80001,1,1,16018,0.55,GGG,ACGACAATCCGGTGATCAGC,TCCGGTGATCAGC,0
2541,NC_003888.3,region1,80001,1,1,16017,0.50,CGG,TACGACAATCCGGTGATCAG,ATCCGGTGATCAG,0
...,...,...,...,...,...,...,...,...,...,...,...
191,NC_003888.3,region1,80001,1,-1,1988,0.65,CGG,ATCCTCGCCGTGACCGTCAC,CCGTGACCGTCAC,7
172,NC_003888.3,region1,80001,1,-1,1765,0.65,TGG,TGGTAGAGCGCAGCATCGGC,GCGCAGCATCGGC,8
3954,NC_003888.3,region1,80001,1,1,2235,0.70,GGG,AGTAGACGGTGCCGTGGCCG,GGTGCCGTGGCCG,8
179,NC_003888.3,region1,80001,1,-1,1842,0.70,AGG,CACCCTGGGCATCGACGTCG,GGCATCGACGTCG,8


In [6]:
# Filter the DataFrame to retain only up to 5 sgRNA sequences per locus_tag
filtered_df = sgrna_df.groupby('locus_tag').head(number_of_sgRNAs_per_group)
filtered_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
3517,NC_003888.3,region1,80001,1,1,6574,0.60,TGG,GCCACATTCACTACGACCCC,TCACTACGACCCC,0
2538,NC_003888.3,region1,80001,1,1,16046,0.65,GGG,CCCTGATCCGAGTGTGTGCC,CCGAGTGTGTGCC,0
5691,NC_003888.3,region3,9001,1,1,3477,0.70,GGG,GCCGGTCGGCCATATCCCCT,GGCCATATCCCCT,1
7003,NC_003888.3,region4,15001,1,1,265,0.65,AGG,ACGGCGTACAGGTCGCGTAC,ACAGGTCGCGTAC,1
4772,NC_003888.3,region2,4001,1,1,346,0.70,TGG,GTCCCTCGACTCACTGGCGC,GACTCACTGGCGC,1
5251,NC_003888.3,region3,9001,1,-1,2460,0.40,TGG,TAGTTGTGTGGGATTGATCT,GTGGGATTGATCT,1
4773,NC_003888.3,region2,4001,1,1,340,0.65,TGG,GTGCTGGTCCCTCGACTCAC,TCCCTCGACTCAC,1
7004,NC_003888.3,region4,15001,1,1,254,0.65,AGG,TCACGTCACCGACGGCGTAC,ACCGACGGCGTAC,1


## Output

In [7]:
# MAke oligoes
list_of_ssDNAs = make_ssDNA_oligos(filtered_df, upstream_ovh = up_homology,
                      downstream_ovh=dw_homology)
print(list_of_ssDNAs[0].name)

# cut plasmid
from Bio.Restriction import NcoI
linearized_plasmid = sorted(clean_plasmid.cut(NcoI), key=lambda x: len(x), reverse=True)[0]
#print(linearized_plasmid)

sgRNA_vectors = assemble_plasmids_by_ssDNA_bridging(list_of_ssDNAs,linearized_plasmid)
sgRNA_vectors

region1_loc_6574


[Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279)]

In [8]:

# Constructing a meaningful name, ID, and description for the assembled plasmid using user input
targeting_info = []
for index, row in filtered_df.iterrows():
    formatted_str = f"pCas9_{row['locus_tag']}({row['sgrna_loc']})"
    targeting_info.append(formatted_str)

for i in range(len(sgRNA_vectors)):
    sgRNA_vectors[i].name = f'{targeting_info[i]}'
    sgRNA_vectors[i].id = sgRNA_vectors[i].name  
    sgRNA_vectors[i].description = f'CRISPR-Cas9 targeting {", ".join(genes_to_KO)} for single gene knockout, assembled using StreptoCAD.'


In [9]:
print_plasmids = False

if print_plasmids: 
    for vector in sgRNA_vectors: 
        vector.write(f"../../data/plasmids/sgRNA_plasmids_pCRISPR-Cas9/{vector.id}.gb")

### IDT primers

In [10]:
idt_primers=primers_to_IDT(list_of_ssDNAs)
idt_primers

,Name,Sequence,Concentration,Purification
0,region1_loc_6574,CGGTTGGTAGGATCGACGGCGCCACATTCACTACGACCCCGTTTTA...,25nm,STD
1,region1_loc_16046,CGGTTGGTAGGATCGACGGCCCCTGATCCGAGTGTGTGCCGTTTTA...,25nm,STD
2,region3_loc_3477,CGGTTGGTAGGATCGACGGCGCCGGTCGGCCATATCCCCTGTTTTA...,25nm,STD
3,region4_loc_265,CGGTTGGTAGGATCGACGGCACGGCGTACAGGTCGCGTACGTTTTA...,25nm,STD
4,region2_loc_346,CGGTTGGTAGGATCGACGGCGTCCCTCGACTCACTGGCGCGTTTTA...,25nm,STD
5,region3_loc_2460,CGGTTGGTAGGATCGACGGCTAGTTGTGTGGGATTGATCTGTTTTA...,25nm,STD
6,region2_loc_340,CGGTTGGTAGGATCGACGGCGTGCTGGTCCCTCGACTCACGTTTTA...,25nm,STD
7,region4_loc_254,CGGTTGGTAGGATCGACGGCTCACGTCACCGACGGCGTACGTTTTA...,25nm,STD


# In frame deletion

In [11]:
in_frame_deletion = True

In [12]:
if in_frame_deletion:
    

    # Make repair templates
    repair_templates_data = find_up_dw_repair_templates(genome, 
                                                        genes_to_KO, 
                                                        target_tm=melting_temperature, 
                                                        
                                                        primer_tm_kwargs={'conc':primer_concentration, 'prodcode':chosen_polymerase} )


    # Digest the plasmids
    processed_records = [Dseqrecord(record, circular=True).cut(StuI)[0] for record in sgRNA_vectors]

    # Rename them appropriately
    for i in range(len(processed_records)):
        processed_records[i].name = sgRNA_vectors[i].name

    # Assembly 
    assembly_data = assemble_multiple_plasmids_with_repair_templates_for_deletion(genes_to_KO, processed_records, 
                                                                                repair_templates_data, 
                                                                                overlap=40)
    # updating the primer names to something systematic.
    update_primer_names(assembly_data)

    # Parse through the primer df
    primer_df = create_primer_df_from_dict(assembly_data)

    # Rename them appropriately
    for i in range(len(processed_records)):
        processed_records[i].name = sgRNA_vectors[i].name

    # Assembly 
    assembly_data = assemble_multiple_plasmids_with_repair_templates_for_deletion(genes_to_KO, processed_records, 
                                                                                repair_templates_data, 
                                                                                overlap=40)
    # updating the primer names to something systematic.
    update_primer_names(assembly_data)
    print(assembly_data)

    # Parse through the primer df
    primer_df = create_primer_df_from_dict(assembly_data)
    unique_df = primer_df.drop_duplicates(keep='first')

    # IDT df
    idt_df = create_idt_order_dataframe(unique_df, concentration="25nm", purification="STD")

    # Contigs
    assembled_contigs = []
    for data in assembly_data: 
        contig_record = data['contig']
        contig_record.id = f"{data['name']}_w_rep"
        contig_record.name = f"{data['name']}_w_rep"
        assembled_contigs.append(contig_record)


    print_plasmids = False

    if print_plasmids:
        # Write to genbank files
        output_directory = "../../data/plasmids/sgRNA_plasmids_pCRISPR-Cas9_with_repair_templates/"
        for contig in assembled_contigs: 
            contig.write(f"{output_directory}/{contig.id}.gb")

[{'gene_name': 'region1', 'name': 'pCas9_region1(6574)', 'contig': Dseqrecord(o13279), 'up_forwar_p_name': 'primer_1', 'up_reverse_p_name': 'primer_2', 'up_forwar_p': primer_1 59-mer:5'-AAGGCCGCTTTTGCG..AGC-3', 'up_reverse_p': primer_2 36-mer:5'-GCCCGGCCGACTCGT..ACG-3', 'up_forwar_primer_str': 'AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGATGACGACTGTGACGAGC', 'up_reverse_primer_str': 'GCCCGGCCGACTCGTTGGGCATCTGCCGGGAGAACG', 'up_forwar_p_anneal': Seq('GATGACGACTGTGACGAGC'), 'up_reverse_p_anneal': Seq('ATCTGCCGGGAGAACG'), 'tm_up_forwar_p': 59, 'tm_up_reverse_p': 57, 'ta_up': 61, 'dw_forwar_p_name': 'primer_3', 'dw_reverse_p_name': 'primer_4', 'dw_forwar_p': primer_3 34-mer:5'-ATGCCGTTCTCCCGG..CGG-3', 'dw_reverse_p': primer_4 53-mer:5'-TCCCCGTCCGGGACC..CCG-3', 'dw_forwar_primer_str': 'ATGCCGTTCTCCCGGCAGATGCCCAACGAGTCGG', 'dw_reverse_primer_str': 'TCCCCGTCCGGGACCCGCGCGGTCGATCCCCGCATATAGGCGGCGAGAGGCCG', 'dw_forwar_p_anneal': Seq('GCCCAACGAGTCGG'), 'dw_reverse_p_anneal': Seq('CGGCGAGAGGCCG'), 'tm

In [13]:
workflow_df = determine_workflow_order_for_plasmids(sgRNA_vectors, 
                                                            assembled_contigs,
                                                            ["StuI", "NcoI"])
workflow_df

,sgRNA plasmid,repair template plasmid,which workflow to proceed with,sgRNA plasmid #StuI sites,repair template plasmid #NcoI sites
0,pCas9_region1(6574),pCas9_region1(6574)_w_rep,Proceed with sgRNA integration first,1,0
1,pCas9_region1(16046),pCas9_region1(16046)_w_rep,Proceed with sgRNA integration first,1,0
2,pCas9_region3(3477),pCas9_region3(3477)_w_rep,Proceed with sgRNA integration first,1,0
3,pCas9_region4(265),pCas9_region4(265)_w_rep,Proceed with sgRNA integration first,1,0
4,pCas9_region2(346),pCas9_region2(346)_w_rep,Proceed with sgRNA integration first,1,0
5,pCas9_region3(2460),pCas9_region3(2460)_w_rep,Proceed with sgRNA integration first,1,0
6,pCas9_region2(340),pCas9_region2(340)_w_rep,Proceed with sgRNA integration first,1,0
7,pCas9_region4(254),pCas9_region4(254)_w_rep,Proceed with sgRNA integration first,1,0


In [14]:
if in_frame_deletion:
    integration_names = filtered_df.apply(lambda row: f"sgRNA_{row['locus_tag']}({row['sgrna_loc']})", axis=1).tolist()
    plasmid_metadata_df = extract_metadata_to_dataframe(assembled_contigs,
                                                        clean_plasmid,
                                                        integration_names)
    workflow_df = determine_workflow_order_for_plasmids(sgRNA_vectors, 
                                                            assembled_contigs,
                                                            ["StuI", "NcoI"])
    
    print(f'Determining the order of workflows : {workflow_df}')

else: 
    integration_names = filtered_df.apply(lambda row: f"sgRNA_{row['locus_tag']}({row['sgrna_loc']})", axis=1).tolist()
    plasmid_metadata_df = extract_metadata_to_dataframe(sgRNA_vectors,
                                                        clean_plasmid,
                                                        integration_names)

    plasmid_metadata_df

Determining the order of workflows :           sgRNA plasmid     repair template plasmid  \
0   pCas9_region1(6574)   pCas9_region1(6574)_w_rep   
1  pCas9_region1(16046)  pCas9_region1(16046)_w_rep   
2   pCas9_region3(3477)   pCas9_region3(3477)_w_rep   
3    pCas9_region4(265)    pCas9_region4(265)_w_rep   
4    pCas9_region2(346)    pCas9_region2(346)_w_rep   
5   pCas9_region3(2460)   pCas9_region3(2460)_w_rep   
6    pCas9_region2(340)    pCas9_region2(340)_w_rep   
7    pCas9_region4(254)    pCas9_region4(254)_w_rep   

         which workflow to proceed with  sgRNA plasmid #StuI sites  \
0  Proceed with sgRNA integration first                          1   
1  Proceed with sgRNA integration first                          1   
2  Proceed with sgRNA integration first                          1   
3  Proceed with sgRNA integration first                          1   
4  Proceed with sgRNA integration first                          1   
5  Proceed with sgRNA integration first        

In [15]:
# Getting checking primers
checking_primers_df = find_best_check_primers_from_genome(genome, 
                                       genes_to_KO, 
                                       flanking_region=flanking_region,
                                       target_tm = melting_temperature, 
                                        primer_concentration = primer_concentration, 
                                        polymerase = chosen_polymerase)
checking_primers_df


,locus tag,f_primer_name,r_primer_name,f_primer_sequences(5-3),r_primer_sequences(5-3),f_tm,r_tm,ta,flanking_region,annealing_temperature,...,homodimer_reverse_tm,homodimer_reverse_deltaG (kcal/mol),heterodimer_tm,heterodimer_deltaG (kcal/mol),hairpin_forward_structure_found,hairpin_forward_tm,hairpin_forward_deltaG (kcal/mol),hairpin_reverse_structure_found,hairpin_reverse_tm,hairpin_reverse_deltaG (kcal/mol)
0,region2,region2_fwd_checking_primer,region2_rev_checking_primer,GACCGCGCTGGGTG,TCCGGCCGCATCG,59,57,60,500,60,...,22.848051,-5.018625,6.535424,-0.061707,False,0.0,0.0,False,0.0,0.0
1,region4,region4_fwd_checking_primer,region4_rev_checking_primer,TTGCTGCAACCTGCCTTG,TAGGTGCCTCCGTCGA,61,58,62,500,62,...,-13.794142,-0.897729,-1.321907,0.948660,False,0.0,0.0,False,0.0,0.0
2,region3,region3_fwd_checking_primer,region3_rev_checking_primer,ATCCCGCAGCTGCAGG,GAGGAGCGGCAATGGTCATC,61,63,65,500,65,...,1.445721,-1.026208,9.252334,-1.213024,False,0.0,0.0,False,0.0,0.0
3,region1,region1_fwd_checking_primer,region1_rev_checking_primer,GCCGCTGGCCAAGC,AGGCGTGCGCGTGG,60,62,63,700,63,...,22.176370,-2.921991,11.333376,-0.877576,False,0.0,0.0,False,0.0,0.0


In [16]:
# making the primers into dseqrecords
checking_primers_df_idt = create_idt_order_dataframe(checking_primers_df)
checking_primers_df_idt


,Name,Sequence,Concentration,Purification
0,region2_fwd_checking_primer,GACCGCGCTGGGTG,25nm,STD
1,region4_fwd_checking_primer,TTGCTGCAACCTGCCTTG,25nm,STD
2,region3_fwd_checking_primer,ATCCCGCAGCTGCAGG,25nm,STD
3,region1_fwd_checking_primer,GCCGCTGGCCAAGC,25nm,STD
4,region2_rev_checking_primer,TCCGGCCGCATCG,25nm,STD
5,region4_rev_checking_primer,TAGGTGCCTCCGTCGA,25nm,STD
6,region3_rev_checking_primer,GAGGAGCGGCAATGGTCATC,25nm,STD
7,region1_rev_checking_primer,AGGCGTGCGCGTGG,25nm,STD


In [17]:
if in_frame_deletion: 
    full_idt = pd.concat([idt_primers, idt_df, checking_primers_df_idt])
else: 
    full_idt = pd.concat([idt_primers, checking_primers_df_idt])

full_idt

,Name,Sequence,Concentration,Purification
0,region1_loc_6574,CGGTTGGTAGGATCGACGGCGCCACATTCACTACGACCCCGTTTTA...,25nm,STD
1,region1_loc_16046,CGGTTGGTAGGATCGACGGCCCCTGATCCGAGTGTGTGCCGTTTTA...,25nm,STD
2,region3_loc_3477,CGGTTGGTAGGATCGACGGCGCCGGTCGGCCATATCCCCTGTTTTA...,25nm,STD
3,region4_loc_265,CGGTTGGTAGGATCGACGGCACGGCGTACAGGTCGCGTACGTTTTA...,25nm,STD
4,region2_loc_346,CGGTTGGTAGGATCGACGGCGTCCCTCGACTCACTGGCGCGTTTTA...,25nm,STD
5,region3_loc_2460,CGGTTGGTAGGATCGACGGCTAGTTGTGTGGGATTGATCTGTTTTA...,25nm,STD
6,region2_loc_340,CGGTTGGTAGGATCGACGGCGTGCTGGTCCCTCGACTCACGTTTTA...,25nm,STD
7,region4_loc_254,CGGTTGGTAGGATCGACGGCTCACGTCACCGACGGCGTACGTTTTA...,25nm,STD
0,primer_1,AAGGCCGCTTTTGCGGGATCTCGTCGAAGGCACTAGAAGGGATGAC...,25nm,STD
1,primer_3,ATGCCGTTCTCCCGGCAGATGCCCAACGAGTCGG,25nm,STD


## Folder with all the generated I/O

In [18]:
input_files = [
    {"name": "input_genome.gb", "content": genome},
    {"name": "input_plasmid.gb", "content": clean_plasmid}
]

if in_frame_deletion: 
    output_files = [
        {"name": "cBEST_w_sgRNAs.gb", "content": assembled_contigs}, # LIST OF Dseqrecords
        {"name": "primer_df.csv", "content": primer_df},
        {"name": "full_idt.csv", "content": full_idt},
        {"name": "sgrna_df.csv", "content": sgrna_df},
        {"name": "filtered_df.csv", "content": filtered_df},
        {"name": "plasmid_metadata_df.csv", "content": plasmid_metadata_df},

    ]
else: 
    output_files = [
        {"name": "cBEST_w_sgRNAs.gb", "content": sgRNA_vectors}, # LIST OF Dseqrecords
        {"name": "full_idt.csv", "content": full_idt},
        {"name": "sgrna_df.csv", "content": sgrna_df},
        {"name": "filtered_df.csv", "content": filtered_df},
        {"name": "plasmid_metadata_df.csv", "content": plasmid_metadata_df},
        {"name": "workflow_order_df.csv", "content": workflow_df},

        workflow_df

    ]

input_values = {
    "genes_to_knockout": genes_to_KO,


    "filtering_metrics": {
        "gc_upper": gc_upper,
        "gc_lower": gc_lower,
        "off_target_seed": off_target_seed,
        "off_target_upper": off_target_upper,
        "cas_type": cas_type,
        "number_of_sgRNAs_per_group": number_of_sgRNAs_per_group,

    },  
    "polymerase_settings": {
        "chosen_polymerase": chosen_polymerase,
        "melting_temperature": melting_temperature,
        "primer_concentration": primer_concentration,
        "primer_number_increment": primer_number_increment,
        "flanking_region": flanking_region
    },
    "overlapping_sequences": {
        "up_homology": str(up_homology),
        "dw_homology": str(dw_homology)
    }
}


# Paths to Markdown files
markdown_file_paths = [
    "../../protocols/conjugation_protcol.md",
    "../../protocols/single_target_crispr_plasmid_protcol.md"

]


timestamp = datetime.utcnow().isoformat()

# Create project directory structure
project_directory = ProjectDirectory(
    project_name=f"CRISPR_cas9_plasmid_workflow_{timestamp}",
    input_files=input_files,
    output_files=output_files,
    input_values=input_values,
    markdown_file_paths=markdown_file_paths
)

# DO You want to save the folder? 
save_zip_folder = False 

if save_zip_folder: 
    # Generate the project directory structure and get the zip content
    zip_content = project_directory.create_directory_structure(create_directories=False)

    # Save the zip file to disk (optional)
    with open("project_structure.zip", "wb") as f:
        f.write(zip_content)